In [11]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
from torch import nn, optim

In [12]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Update dataset path
dataset_path = '/Users/evancoons/Downloads/leafsnap-dataset/dataset/images/field'

# Load the dataset
dataset = datasets.ImageFolder(dataset_path, transform=transform)

# Splitting the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
dataset

Dataset ImageFolder
    Number of datapoints: 7719
    Root location: /Users/evancoons/Downloads/leafsnap-dataset/dataset/images/field
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [14]:
# Load a pretrained ResNet50 model
#model = models.resnet50(weights=True)
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Freeze all layers in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a new one with the appropriate number of output classes
num_classes = len(dataset.classes)  # Number of classes in the dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/evancoons/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [02:13<00:00, 766kB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Only train the last layer

In [17]:
# Define number of epochs
num_epochs = 10

# Start training
model.train()
for module in model.modules():
    if isinstance(module, nn.BatchNorm2d):
        module.eval()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 1.7328966856002808
Epoch 2, Loss: 1.0150320529937744
Epoch 3, Loss: 0.49588292837142944
Epoch 4, Loss: 0.4636046886444092
Epoch 5, Loss: 0.39162537455558777
Epoch 6, Loss: 0.23318715393543243
Epoch 7, Loss: 0.26274439692497253
Epoch 8, Loss: 0.1846999228000641
Epoch 9, Loss: 0.20317381620407104
Epoch 10, Loss: 0.10773222148418427


In [16]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


number of labels:  32
amount correct:  0
example:  tensor(21) tensor(29)
number of labels:  32
amount correct:  0
example:  tensor(94) tensor(180)
number of labels:  32
amount correct:  0
example:  tensor(145) tensor(38)


KeyboardInterrupt: 

In [18]:
model_path = '/Users/evancoons/Downloads/model2.pth'  # Specify the path to save your model
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

Model saved to /Users/evancoons/Downloads/model2.pth
